## Tuning Models

In [1]:
%load_ext autoreload
%matplotlib inline
# always reload modules marked with "%aimport"
%autoreload 1

In [2]:
#load stored data
%store -r preprocess_data
%store -r group_df
%store -r ili_h1

In [9]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from keras.layers import LSTM

from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

In [10]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [11]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [12]:
%store -r reframed
# split into train and test sets
values = reframed.values
n_train_weeks = 5*20*52

In [13]:
train = values[:n_train_weeks, :]
test = values[n_train_weeks:, :]

In [14]:
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [15]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

((5200, 1, 8), (5200,), (5938, 1, 8), (5938,))


In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense

# fix random seed for reproducibility
seed = 155
np.random.seed(seed)

In [17]:
train_X.shape[2]

8

In [18]:
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, 8)))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [20]:
%%time
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100, 200, 500, 1000]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(train_X, train_y)

/Users/bbuildman/anaconda3/envs/python2/lib/python2.7/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.122381). Check your callbacks.
  % delta_t_median)


CPU times: user 12min 19s, sys: 1min 58s, total: 14min 18s
Wall time: 11h 12min 36s


In [21]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.774231 using {'epochs': 1000, 'batch_size': 40}
0.768269 (0.006861) with: {'epochs': 10, 'batch_size': 10}
0.771154 (0.006063) with: {'epochs': 50, 'batch_size': 10}
0.773846 (0.007815) with: {'epochs': 100, 'batch_size': 10}
0.773077 (0.008776) with: {'epochs': 200, 'batch_size': 10}
0.771154 (0.006208) with: {'epochs': 500, 'batch_size': 10}
0.735385 (0.024115) with: {'epochs': 1000, 'batch_size': 10}
0.768462 (0.007149) with: {'epochs': 10, 'batch_size': 20}
0.770769 (0.006025) with: {'epochs': 50, 'batch_size': 20}
0.772885 (0.007676) with: {'epochs': 100, 'batch_size': 20}
0.773846 (0.008631) with: {'epochs': 200, 'batch_size': 20}
0.774038 (0.008505) with: {'epochs': 500, 'batch_size': 20}
0.771538 (0.007032) with: {'epochs': 1000, 'batch_size': 20}
0.761731 (0.012073) with: {'epochs': 10, 'batch_size': 40}
0.769423 (0.006458) with: {'epochs': 50, 'batch_size': 40}
0.770769 (0.006387) with: {'epochs': 100, 'batch_size': 40}
0.773077 (0.008386) with: {'epochs': 200, 'batch

## Tune Learning Rate and Momentum

In [22]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, 8)))
    model.add(Dense(1))
    # Compile model
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='mae', optimizer=optimizer, metrics=['accuracy'])
    return model

In [23]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

In [24]:
# define the grid search parameters
%%time
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(train_X, train_y)

SyntaxError: invalid syntax (<ipython-input-24-7f9476248ef1>, line 2)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))